<a href="https://colab.research.google.com/github/dkv1234/QKD-BB84/blob/main/QKD(BB84_Protocol).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 7.0 MB/s eta 0:00:00


In [ ]:
!pip install qiskit qiskit-aer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 65.9 MB/s eta 0:00:00


In [ ]:
# Import necessary modules
from qiskit import QuantumCircuit
from qiskit.primitives import Sampler
import numpy as np

# Initialize variables
key_length = 100  # Length of the key to be generated
alice_key = []  # Alice's key
bob_key = []  # Bob's key
alice_bases = []  # Alice's random bases
bob_bases = []  # Bob's random bases

# Step 1: Alice's Qubit Preparation and Sending
# Function to generate Alice's random key bits and bases
def alice_prepare_qubits():
    for _ in range(key_length):
        # Generate random bits and bases for Alice
        alice_bit = np.random.randint(2)  # 0 or 1
        alice_base = np.random.choice(['X', 'Z'])  # X or Z basis
        alice_key.append(alice_bit)
        alice_bases.append(alice_base)
    return alice_key, alice_bases

# Step 2: Simulate Quantum Transmission and Measurement (Alice -> Bob)
def bob_measure_qubits():
    sampler = Sampler()  # Initialize the Sampler primitive

    for i in range(key_length):
        # Bob chooses a random basis
        bob_base = np.random.choice(['X', 'Z'])
        bob_bases.append(bob_base)

        # Prepare a quantum circuit to send qubits
        qc = QuantumCircuit(1, 1)
        if alice_key[i] == 1:
            qc.x(0)  # Apply X gate if bit is 1
        if alice_bases[i] == 'X':
            qc.h(0)  # Apply H gate for X-basis preparation

        # Apply measurement based on Bob's choice of basis
        if bob_bases[i] == 'X':
            qc.h(0)
        qc.measure(0, 0)

        # Run the circuit using Sampler
        result = sampler.run(circuits=[qc]).result()
        counts = result.quasi_dists[0]
        measured_bit = 0 if counts.get(0, 0) > counts.get(1, 0) else 1
        bob_key.append(measured_bit)

# Step 3: Basis Comparison (Classical Communication Phase)
def basis_comparison():
    matching_bases = []
    matching_bits = []

    for i in range(key_length):
        if alice_bases[i] == bob_bases[i]:  # Keep bits with matching bases
            matching_bases.append(i)
            matching_bits.append(alice_key[i])

    # Calculate the raw key after basis matching
    raw_key_alice = [alice_key[i] for i in matching_bases]
    raw_key_bob = [bob_key[i] for i in matching_bases]

    # Compare and output the matched key indices
    print(f'Matching bases indices: {matching_bases}')
    print(f'Raw Key Alice: {raw_key_alice}')
    print(f'Raw Key Bob: {raw_key_bob}')
    return raw_key_alice, raw_key_bob

# Step 4: Error Detection (Classical Communication Phase)
def error_detection(raw_key_alice, raw_key_bob):
    # Publicly compare a portion of the key to check for eavesdropping
    sample_size = int(len(raw_key_alice) * 0.2)
    sample_indices = np.random.choice(len(raw_key_alice), sample_size, replace=False)

    discrepancies = sum(1 for i in sample_indices if raw_key_alice[i] != raw_key_bob[i])
    print(f'Number of discrepancies: {discrepancies}')

    if discrepancies > 0:
        print("Eavesdropping detected! Key discarded.")
    else:
        print("No discrepancies detected. Key secure.")

# Run each step of the protocol
alice_key, alice_bases = alice_prepare_qubits()
bob_measure_qubits()
raw_key_alice, raw_key_bob = basis_comparison()
error_detection(raw_key_alice, raw_key_bob)


Matching bases indices: [0, 5, 7, 9, 10, 11, 14, 15, 16, 17, 21, 23, 25, 26, 27, 28, 29, 31, 32, 33, 34, 36, 38, 40, 41, 44, 49, 51, 52, 53, 57, 58, 66, 70, 72, 73, 76, 78, 80, 81, 82, 83, 84, 85, 87, 89, 90, 93, 94, 95, 98]
Raw Key Alice: [1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1]
Raw Key Bob: [1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1]
Number of discrepancies: 0
No discrepancies detected. Key secure.


<ipython-input-11-2c3b40e344d3>:26: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()  # Initialize the Sampler primitive
